In [1]:
import pandas as pd
import numpy as np
import torch
from SoPred.march.dnn import SoPredModel
from RtPred.march.neumf import RtPredModel
from Inference.dataset import InferenceDataset
from torch.utils.data import DataLoader

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
testfile = 'dataset/test.csv'
test_df = pd.read_csv(testfile)

n_users = test_df['reviewerID'].max() + 1
n_items = test_df['asin'].max() + 1

model1_pth = 'models/best_model1.pt'
model2_pth = 'models/best_model2.pt'

model1 = SoPredModel(n_users, n_items).to(DEVICE)
model1.load_state_dict(torch.load(model1_pth)['model_state_dict'])
model1.eval()

model2 = RtPredModel(n_users, n_items).to(DEVICE)
model2.load_state_dict(torch.load(model2_pth)['model_state_dict'])
model2.eval()

/tmp/ipykernel_23143/671842866.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load(model1_pth)['model_state_dict'])


RuntimeError: Error(s) in loading state_dict for SoPredModel:
	Missing key(s) in state_dict: "user_embedding.weight", "item_embedding.weight", "fc4.weight", "fc4.bias". 
	Unexpected key(s) in state_dict: "mf_usr_emb.weight", "mf_item_emb.weight", "nn_usr_emb.weight", "nn_item_emb.weight", "neumf.weight", "neumf.bias". 
	size mismatch for fc1.weight: copying a param with shape torch.Size([128, 128]) from checkpoint, the shape in current model is torch.Size([512, 256]).
	size mismatch for fc1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([512]).
	size mismatch for fc2.weight: copying a param with shape torch.Size([64, 128]) from checkpoint, the shape in current model is torch.Size([256, 512]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for fc3.weight: copying a param with shape torch.Size([32, 64]) from checkpoint, t

In [3]:
items = test_df['asin'].unique()
users = test_df['reviewerID'].value_counts()
users = users[users > 10].index
sampled_users = set(np.random.choice(users, 1000, replace=False))

user_items = {}

for user, value in test_df.groupby('reviewerID'):
    if user in sampled_users:
        user_items[user] = list(zip(value['asin'].values, value['overall'].values))

In [ ]:
for user in sampled_users:
    dataset = InferenceDataset(user, items)
    dataloader = DataLoader(dataset, batch_size=1024, shuffle=False)
    
    for batch in dataloader:
        user = batch['user_id'].to(DEVICE)
        item = batch['item_id'].to(DEVICE)
        
        score = model1(user, item).unsqueeze(-1)
        
        pred = model2(user, item, score)
        
        pred = torch.argmax(pred, dim=-1) + 1

KeyboardInterrupt: 